In [2]:
import pandas as pd

df=pd.read_csv(r"C:\Users\shrav\OneDrive\Desktop\proj\Customer_Behavior_Analysis\customer_shopping_behavior.csv")
df

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [4]:
df.columns = [c.strip().replace(" ", "_").lower() for c in df.columns]


In [5]:
df = df.rename(columns={"purchase_amount_(usd)": "purchase_amount_usd"})

In [6]:
df['purchase_amount_usd'] = pd.to_numeric(df['purchase_amount_usd'], errors='coerce')
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')
df['previous_purchases'] = pd.to_numeric(df['previous_purchases'], errors='coerce')

In [7]:
df['gender'] = df['gender'].str.title()
df['category'] = df['category'].str.title().str.strip()
df['shipping_type'] = df['shipping_type'].str.title().str.strip()
df['subscription_status'] = df['subscription_status'].fillna("No").str.title()


In [8]:
freq_map = {
    'Weekly': 7,
    'Fortnightly': 14,
    'Bi-Weekly': 14,
    'Monthly': 30,
    'Quarterly': 90,
    'Annually': 365
}
df['purchase_freq_days'] = df['frequency_of_purchases'].map(freq_map)

In [9]:
df['discount_flag'] = df['discount_applied'].str.lower().map({'yes': 1, 'no': 0})
df['promo_flag'] = df['promo_code_used'].str.lower().map({'yes': 1, 'no': 0})

In [10]:
df['discount_flag'] = df['discount_flag'].fillna(0).astype(int)
df['promo_flag'] = df['promo_flag'].fillna(0).astype(int)

In [11]:
cust = df.groupby('customer_id').agg(
    total_spent=('purchase_amount_usd', 'sum'),
    avg_spent=('purchase_amount_usd', 'mean'),
    purchase_count=('purchase_amount_usd', 'count'),
    avg_rating=('review_rating', 'mean'),
    last_prev_purchases=('previous_purchases', 'max')
).reset_index()

In [12]:
df.to_csv("clean_transactions.csv", index=False)
cust.to_csv("customers_aggregated.csv", index=False)

In [13]:
nulls = df.isnull().sum()
print(nulls[nulls > 0])


review_rating          37
purchase_freq_days    584
dtype: int64


In [14]:
df['review_rating'] = df['review_rating'].fillna(df['review_rating'].mean())


In [15]:
df[df['purchase_freq_days'].isnull()]['frequency_of_purchases'].value_counts()


frequency_of_purchases
Every 3 Months    584
Name: count, dtype: int64

In [16]:
freq_map = {
    'Weekly': 7,
    'Fortnightly': 14,
    'Bi-Weekly': 14,
    'Biweekly': 14,
    'Monthly': 30,
    'Quarterly': 90,
    'Every 3 Months': 90,
    'Annually': 365
}


In [17]:
df['purchase_freq_days'] = df['frequency_of_purchases'].map(freq_map)


In [18]:
df = df.drop(columns=['discount_flag', 'promo_flag'])


In [19]:
df.head(5)

,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,purchase_freq_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,365


In [20]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import pandas as pd


username = "root"
password = "Shvetang@24"
host = "localhost"
port = "3306"
database = "customer_behavior"

password_encoded = quote_plus(password)

engine = create_engine(f"mysql+pymysql://{username}:{password_encoded}@{host}:{port}/")

engine_db = create_engine(f"mysql+pymysql://{username}:{password_encoded}@{host}:{port}/{database}")

table_name = "customer"
df.to_sql(table_name, engine_db, if_exists="replace", index=False)
print("Data uploaded successfully!")


Data uploaded successfully!


In [21]:
df.to_excel("cleaned_data.xlsx", index=False)
